In [16]:
from modules.Scrape import RaceScrapeer
import requests
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO

In [2]:
r = RaceScrapeer

In [4]:
r.login_info

AttributeError: type object 'RaceScrapeer' has no attribute 'login_info'

In [6]:
user_id = "ken.osechi@gmail.com"
password = "BQj4WGwzLxJZNkWiw"
login_info = {
    "login_id": user_id,
    "pswd": password,
}
session = requests.session()
url_login = "https://regist.netkeiba.com/account/?pid=login&action=auth"
ses = session.post(url_login, data=login_info)
cookie = ses.cookies

In [31]:
race_id = "202301090112"

In [37]:
url = "https://db.sp.netkeiba.com/race/202101010101"
html = session.get(url, cookies=cookie)
html.encoding = "EUC-JP"
soup = BeautifulSoup(html.content, "html.parser")
race_data = soup.find("div", class_="RaceData").find_all("span")
race_data = "".join(tag.get_text() for tag in race_data)

In [38]:
pd.read_html(StringIO(html.text))

[    着順  枠 番  馬 番         馬名  性齢  斤量    騎手     タイム     着差  タイム 指数  ...    上り  \
 0    1    5    9  ディーバサンライズ  牝3  52  秋山稔樹  1:08.7    NaN      87  ...  34.6   
 1    2    3    6      バンベルク  牡3  56  横山和生  1:08.8    3/4      85  ...  35.0   
 2    3    7   14      リツィタル  牝3  54  大野拓弥  1:08.9    1/2      84  ...  34.0   
 3    4    5   10  クレマチステソーロ  牝3  54  鮫島克駿  1:09.0    1/2      83  ...  35.2   
 4    5    3    5  ペイシャケイティー  牝3  54  古川吉洋  1:09.0     ハナ      83  ...  34.5   
 5    6    1    2       セリシア  牝3  54    武豊  1:09.0     クビ      83  ...  34.9   
 6    7    4    7      デクラーク  牡3  56  横山武史  1:09.2      1      80  ...  33.9   
 7    8    6   11    プレジャジンク  牝3  54  吉田隼人  1:09.2    アタマ      80  ...  35.3   
 8    9    7   13     メルティング  牝3  54  団野大成  1:09.6  2.1/2      74  ...  34.5   
 9   10    2    3    リトルロータス  牝3  52  山田敬士  1:09.6    アタマ      74  ...  34.5   
 10  11    4    8    ニースバローズ  牝3  51  横山琉人  1:09.7     クビ      73  ...  34.9   
 11  12    2    4    マイネルニゲラ  牡3  56  丹内

In [ ]:
a = pd.read_html(StringIO(html.text))[0]
a

In [ ]:
df = pd.read_pickle("horse_results/Race_2021_0-1000.pickle")
df

In [39]:
from modules.Scrape import RaceScrapeer

# RaceScrapeerのインスタンスを作成
scraper = RaceScrapeer()

# cookieの値を確認
print(scraper.cookie)

<RequestsCookieJar[]>


In [ ]:
import requests
from bs4 import BeautifulSoup

import login_info


class NetkeibaLogin:
    def __init__(self, login_info):
        self.login_info = {
            "login_id": login_info.USERID,
            "pswd": login_info.PASSWORD,
        }
        self.session = requests.session()
        self.url_login = "https://regist.netkeiba.com/account/?pid=login&action=auth"
        self.headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        }

    def login(self):
        # First, get the login page to check for any CSRF token
        login_page = self.session.get(self.url_login, headers=self.headers)
        soup = BeautifulSoup(login_page.content, "html.parser")

        # Check for any hidden input fields and add them to login_info
        for hidden_input in soup.find_all("input", type="hidden"):
            self.login_info[hidden_input["name"]] = hidden_input["value"]

        # Attempt login
        self.ses = self.session.post(
            self.url_login,
            data=self.login_info,
            headers=self.headers,
            allow_redirects=False,
        )

        if self.ses.status_code == 302:
            redirect_url = self.ses.headers["Location"]
            self.ses = self.session.get(redirect_url, headers=self.headers)

        # Verify login success
        test_url = (
            "https://regist.netkeiba.com/account/"  # Replace with appropriate URL
        )
        test_response = self.session.get(test_url, headers=self.headers)

        if "ログアウト" in test_response.text or "マイページ" in test_response.text:
            print("Login successful")
        else:
            print("Login failed")

        print(f"Cookies: {self.session.cookies}")

        return self.session


# Usage
# login_info = YourLoginInfoClass()  # Replace with your actual login info class
netkeiba = NetkeibaLogin(login_info)
logged_in_session = netkeiba.login()